# Lab03: Web Crawler (Continue) & Information Retrieval.

- MSSV: 19120138
- Họ và tên: Trần Đức Thuỵ

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `t_data.txt` và file `raw_data` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập dữ liệu từ: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

Cài đặt một Web crawler đơn giản bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining, tìm liên kết và thu thập dữ liệu trong HTML tại URL này sau đó lặp lại với các URL vừa tìm được.


In [116]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle
import sys

## 1. Thu thập đường dẫn

- Robot.txt:  https://en.wikipedia.org/robots.txt
- **Bước 1**: Thu thập đường dẫn từ https://en.wikipedia.org/wiki/Web_mining. Lưu trữ vào một danh sách `url_list`. 
- **Bước 2**: Lặp lại bước 1 cho các đường dẫn trong `url_list` (**lưu ý:** kiểm tra các đường dẫn vừa thu được đã nằm trong `url_list` hay không?). Dừng khi đã thu thập được 200 URLs.

In [117]:
def get_urls(url):
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    r = requests.get(url)
    text = r.text
    url_regex =r"http[s]*://[a-zA-Z0-9?&-=;/_]*"
    urls = re.findall(url_regex, text)
    urls = list(set(urls))
    urls.sort(key=lambda k: len(k))
    urls = [url.strip("');") for url in urls]
    return urls

def get_urls_recursive(start_url, limit):
    urls = [start_url]
    for url in urls:
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        new_url = get_urls(url)
        urls.extend(new_url)
        urls = list(set(urls))
        if(len(urls)>limit):
            break
    return urls[:limit]
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 200)

In [119]:
#Kiem tra do dai cua url_list
len(url_list)

200

## 2. Thu thập dữ liệu
Thu thập dữ liệu từ `url_list`. Lưu trữ dữ liệu thu được vào dictionary data với keys là các từ, values gồm 2 phần tử: 
- `url_idx_list` với $idx \in \left[0,200\right) \cap \mathbb{N}$
- `frequency` 
    
Ví dụ: `data['at']=[url_idx_list,frequency]`:
- `url_idx_list`: danh sách các url mà trong dữ liệu của chúng (html document) chứa từ "at". 
- `frequency`: tần suất xuất hiện (số lần xuất hiện) của từ `at` trong dữ liệu của **tất cả đường dẫn thu được**.   

In [120]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    # TODO
    # Cài đặt lại như Lab02
    try:
        r = requests.get(url)
    except:
        return
    try:
        soup = BeautifulSoup(r.content, "html.parser",from_encoding="iso-8859-1")
        text = soup.findAll(text=True)
        filtered_text = list(filter(text_filter, text)) # list của các chuỗi
        word_list = []
        for i in range(len(filtered_text)):
            for remove_character in string.punctuation:  
                filtered_text[i] = filtered_text[i].replace(remove_character, " ")
            filtered_list = filtered_text[i].split(" ")
            word_list.extend(filtered_list)
        word_list = [i for i in word_list if i!='']
        return word_list
    except:
        return

def read_url(url, url_idx, data):
    # TODO
    # Cài đặt lại như Lab02
    word_list = wordList(url)
    if(word_list!=None):
        for word in word_list:
            if (word not in data.keys()):
                data[word] = [[url_idx], 1]
            else:
                if (url_idx not in data[word][0]):
                    data[word][0].append(url_idx)
                data[word][1] = data[word][1] + 1

In [121]:
data = {}
for url_index, url in enumerate(url_list, 1):
    read_url(url, url_index, data)

## 3. Tiền xử lý
Loại bỏ các item trong data mà key là các stopword.

**Ngữ liệu:** 

In [122]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
print(english_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [123]:
# TODO
# Loại bỏ các key của biến data mà nằm trong danh sách english_stopwords
for word in english_stopwords:
    if(word in data.keys()):
        data.pop(word)
# Kiểm tra xem còn các stopwords trong data hay không
result = set(english_stopwords).intersection(set(data.keys()))
result == set()

True

In [124]:
# Chuẩn hoá các key viết hoa trong dictionary ex: "Web" -> "web"
def lowercase_dict(dictionary):
    return dict((k.lower(), v) for k, v in dictionary.items())

In [125]:
data = lowercase_dict(data)

## 4. Lưu trữ và biểu diễn dữ liệu
Sử dụng pickle lưu lại data với tên file raw_data.
### 4.1 Cơ sở dữ liệu giao tác:
Thông thường, các cơ sở dữ liệu giao tác được lưu trong flat files (các tập phẳng) thay vì trong một hệ cơ sở dữ liệu. Các item là các số nguyên không âm, mỗi giao tác tương ứng với một dòng các số nguyên phân tách nhau bằng khoảng trắng.
Ví dụ:

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 

32 69 

48 70 71 72 

39 73 74 75 76 77 78 79 

36 38 39 41 48 79 80 81 

82 83 84 

41 85 86 87 88 

39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 

36 38 39 48 89 

39 41 102 103 104 105 106 107 108 

38 39 41 109 110 

39 111 112 113 114 115 116 117 118 

119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 

48 134 135 136 

39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 

39 150 151 152 

38 39 56 153 154 155 

In [127]:
with open('raw_data', 'wb') as f:
    # TODO
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

### 4.2 Xuất dataset
Lưu một cơ sở dữ liệu giao tác (transactional database) vào file t_data.txt: 
- Các item tương ứng với url_idx
- Mỗi transaction tương ứng với một từ.

In [129]:
with open('t_data.txt', 'w') as f:
    for word, (url, freq) in data.items():
        print(*url, file=f)

## 5. Truy vấn and, or, not
Ví dụ: 
- Truy vấn `and` câu `web mining`: trả về đường dẫn tới các trang web có cả 2 từ web và từ mining. 
- Truy vấn `or` câu `web mining`: trả về đường dẫn tới các trang web có từ web hoặc từ mining.
- Truy vấn `not` câu `web mining`: trả về đường dẫn tới các trang không có cả từ web và từ mining.

*GỢI Ý: TÁCH CÂU TRUY VẤN THÀNH CÁC TỪ TƯƠNG TỰ PHƯƠNG PHÁP LÀM Ở LAB02.*

In [131]:
ret=[]
def andRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### if len(ret)==0:  return urls
    ### else update ret with urls: intersection of ret and urls 
    keywords = set(sentence.split(' '))
    if (len(keywords)==0 or keywords is None):
        return
    for url_idx, url in enumerate(url_list,1):
        word_in_url = {}
        read_url(url, url_idx, word_in_url)
        set_word = set(word_in_url.keys())
        if (keywords.issubset(set_word)):
            ret.append(url)
    return ret
            
def orRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### find urls have at least 1 term: urls
    ### update ret with urls: extend ret with urls
    keywords = set(sentence.split(' '))
    if (len(keywords)==0 or keywords is None):
        return
    for url_idx, url in enumerate(url_list,1):
        word_in_url = {}
        read_url(url, url_idx, word_in_url)
        set_word = set(word_in_url.keys())
        if (keywords.issubset(set_word)):
            ret.append(url)
        elif keywords.intersection(set_word)!= set():
            ret.append(url)
    return ret

def notRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have at least 1 term: urls
    ### update ret with urls: remove urls from ret
    keywords = set(sentence.split(' '))
    if (len(keywords)==0 or keywords is None):
        return
    for url_idx, url in enumerate(url_list,1):
        word_in_url = {}
        read_url(url, url_idx, word_in_url)
        set_word = set(word_in_url.keys())
        if keywords.intersection(set_word)==set():
            ret.append(url)
    return ret

In [132]:
print(andRetrieval([], 'web mining'))

['https://eu.wikipedia.org/wiki/Datu-meatzaritza', 'http://www.kdnuggets.com/data_mining_course/x1-intro-to-data-mining-notes.html', 'https://en.wikipedia.org/w/index.php?title=Data_mining&amp;oldid=1145629498', 'https://ca.wikipedia.org/wiki/Mineria_de_dades', 'https://en.wikipedia.org/wiki/Web_mining', 'https://archive.org/details/dataminingconcep0000kant', 'https://it.wikipedia.org/wiki/Data_mining', 'https://en.wikipedia.org/wiki/Data_mining']


In [133]:
print(orRetrieval([], 'web mining'))

['https://mn.wikipedia.org/wiki/Data_mining', 'https://eu.wikipedia.org/wiki/Datu-meatzaritza', 'https://foundation.wikimedia.org/wiki/Privacy_policy', 'https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&amp;utm_medium=sidebar&amp;utm_campaign=C13_en.wikipedia.org&amp;uselang=en', 'http://www.kdnuggets.com/data_mining_course/x1-intro-to-data-mining-notes.html', 'https://su.wikipedia.org/wiki/Data_mining', 'http://www.kdnuggets.com/polls/2004/data_mining_methodology.htm', 'https://www.mediawiki.org/wiki/Template:Main_page/bs', 'https://de.wikipedia.org/wiki/Data-Mining', 'https://en.wikipedia.org/w/index.php?title=Data_mining&amp;oldid=1145629498', 'https://www.wikidata.org/wiki/Special:EntityPage/Q172491', 'http://www.kdnuggets.com/polls/2007/data_mining_methodology.htm', 'https://www.doi.org/the-identifier/resources/factsheets/doi-resolution-documentation', 'https://web.archive.org/web/20140609020315/http://www.out-law.com/en/articles/2014/june/researcher

In [134]:
print(notRetrieval([], 'web mining'))

['https://wikis.world/', 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Wikimedia_Hackathon_Prague_2019_-_Group_Photo_-_CLK_-_cropped.jpg/800px-Wikimedia_Hackathon_Prague_2019_-_Group_Photo_-_CLK_-_cropped.jpg', 'http://www.securityfocus.com/brief/277', 'https://curlie.org/Reference/Knowledge_Management/Knowledge_Discovery/Software', 'https://www.mediawiki.org/wiki/Special:MyLanguage/Code_of_Conduct', 'https://doi.org/10.1021', 'https://tr.wikipedia.org/wiki/Veri_madencili', 'https://web.archive.org/web/20130109114939/http://www.iadis.net/dl/final_uploads/200812P033.pdf', 'https://www.mediawiki.org/wiki/Template:Main_page/gu', 'https://zh-yue.wikipedia.org/wiki/', 'https://www.worldcat.org/oclc/45263753', 'https://vi.wikipedia.org/wiki/Khai_ph', 'https://www.mediawiki.org/w/index.php?title=MediaWiki&amp;oldid=3878227', 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Wikimedia_Hackathon_Prague_2019_-_Group_Photo_-_CLK_-_cropped.jpg/640px-Wikimedia_Hackathon_Prague